### Load the trainned prunned model

In [10]:
from qkeras.utils import _add_supported_quantized_objects
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from utils.nn_utils import *

In [11]:
model_dir = 'models'
model_path = f'{model_dir}/quantized_pruned_cnn_model.h5'
# model_path = f'{model_dir}/quantized_svhn.h5'
co = {}
_add_supported_quantized_objects(co)
co['PruneLowMagnitude'] = pruning_wrapper.PruneLowMagnitude


qmodel = tf.keras.models.load_model(model_path, custom_objects=co)
qmodel = strip_pruning(qmodel)

In [12]:
qmodel.summary()

Model: "qkeras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 fused_convbn_0 (QConv2DBat  (None, 26, 26, 8)         113       
 chnorm)                                                         
                                                                 
 conv_act_0 (QActivation)    (None, 26, 26, 8)         0         
                                                                 
 pool_0 (MaxPooling2D)       (None, 13, 13, 8)         0         
                                                                 
 fused_convbn_1 (QConv2DBat  (None, 11, 11, 16)        1233      
 chnorm)                                                         
                                                                 
 conv_act_1 (QActivation)    (None, 11, 11, 16)        0    

In [13]:
reuse_factors_with_serial_pct(qmodel)


Layer: fused_convbn_0 (QConv2DBatchnorm) - Total MACs: 72
 ReuseFactor |  SerialPct
---------------------------
           1 |     0.0139
           2 |     0.0278
           3 |     0.0417
           4 |     0.0556
           6 |     0.0833
           8 |     0.1111
           9 |     0.1250
          12 |     0.1667
          18 |     0.2500
          24 |     0.3333
          36 |     0.5000
          72 |     1.0000

Layer: fused_convbn_1 (QConv2DBatchnorm) - Total MACs: 1152
 ReuseFactor |  SerialPct
---------------------------
           1 |     0.0009
           2 |     0.0017
           3 |     0.0026
           4 |     0.0035
           6 |     0.0052
           8 |     0.0069
           9 |     0.0078
          12 |     0.0104
          16 |     0.0139
          18 |     0.0156
          24 |     0.0208
          32 |     0.0278
          36 |     0.0312
          48 |     0.0417
          64 |     0.0556
          72 |     0.0625
          96 |     0.0833
         128 |    

In [14]:
#project_folder = 'Projects_Reuse_Factor_Analysis'
project_folder = 'ProjectsRF'
project_path = f'{project_folder}/RFanalysis0'
backend = 'Vitis'
default_precision = 'ap_fixed<16,6>'
part = 'xck26-sfvc784-2LV-c'
project_name = 'rf_analysis_0'
#dataset='svhn'
dataset='mnist'
RF=0.0

In [15]:
import hls4ml
import utils.plotting as plotting

# Generate base HLS config
hls_config = hls4ml.utils.config_from_keras_model(
    qmodel,
    granularity='name',
    backend=backend,
    default_precision=default_precision
)

# Set model-level precision
hls_config['Model']['Precision'] = default_precision

# Force Resource strategy globally
for lname, lcfg in hls_config['LayerName'].items():
    lcfg['Strategy'] = 'Resource'

# Inject computed reuse factors
reuse_factors = reuse_percentage_to_factors(qmodel, serial_pct=RF)
for lname, factor in reuse_factors.items():
    if lname in hls_config['LayerName']:
        hls_config['LayerName'][lname]['ReuseFactor'] = factor
        
    else:
        print(f"Warning: Layer {lname} not found in HLS config")

plotting.print_dict(hls_config)


# Convert and compile
hls_model = hls4ml.converters.convert_from_keras_model(
    qmodel,
    hls_config=hls_config,
    backend=backend,
    output_dir=project_path,
    part=part,
    io_type='io_stream',
    clock_period=5,
    trace=True,
    project_name=project_name,
    clock_uncertainty= "12.5"
)
hls_model.compile()


Interpreting Model
Topology:
Layer name: input_4, layer type: InputLayer, input shapes: [[None, 28, 28, 1]], output shape: [None, 28, 28, 1]
Layer name: fused_convbn_0, layer type: QConv2DBatchnorm, input shapes: [[None, 28, 28, 1]], output shape: [None, 26, 26, 8]
Layer name: conv_act_0, layer type: Activation, input shapes: [[None, 26, 26, 8]], output shape: [None, 26, 26, 8]
Layer name: pool_0, layer type: MaxPooling2D, input shapes: [[None, 26, 26, 8]], output shape: [None, 13, 13, 8]
Layer name: fused_convbn_1, layer type: QConv2DBatchnorm, input shapes: [[None, 13, 13, 8]], output shape: [None, 11, 11, 16]
Layer name: conv_act_1, layer type: Activation, input shapes: [[None, 11, 11, 16]], output shape: [None, 11, 11, 16]
Layer name: pool_1, layer type: MaxPooling2D, input shapes: [[None, 11, 11, 16]], output shape: [None, 5, 5, 16]
Layer name: flatten_3, layer type: Reshape, input shapes: [[None, 5, 5, 16]], output shape: [None, 400]
Layer name: dense_0, layer type: QDense, input

/home/theodoros/miniconda3/envs/ai_on_fpga/lib/python3.10/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


In [19]:
from template_injector import TemplateInjector

injector = TemplateInjector(template_dir="templates")

injector.inject(
    project_dir=project_path,
    project_name=project_name,
    force=True  # or False to skip existing files
)

Injected: ProjectsRF/RFanalysis0/firmware/rf_analysis_0_stream.cpp
Injected: ProjectsRF/RFanalysis0/firmware/rf_analysis_0_stream.h
Injected: ProjectsRF/RFanalysis0/rf_analysis_0_test.cpp
Copied: gen_tb.py → ProjectsRF/RFanalysis0/gen_tb.py
Copied: compute_performance.py → ProjectsRF/RFanalysis0/compute_performance.py
Copied: build_prj.tcl → ProjectsRF/RFanalysis0/build_prj.tcl
Copied: golden_preds.py → ProjectsRF/RFanalysis0/golden_preds.py


In [20]:
import os
pwd = os.getcwd()
print(f"Current working directory: {pwd}")
build = True
from utils.build import build_project

Current working directory: /home/theodoros/Projects/AI_ON_FPGA


In [21]:

if build:
    build_project(project_dir=f"{project_path}",
        dataset=dataset,
        n_samples=100
    )


[BUILD] Running: python gen_tb.py --dataset mnist --n-samples 100 --seed 42 --ap-total-bits 16 --ap-int-bits 6 --model keras_model.keras --save-labels
        cwd=/home/theodoros/Projects/AI_ON_FPGA/ProjectsRF/RFanalysis0

[BUILD] Running: vitis-run --mode hls --tcl build_prj.tcl
        cwd=/home/theodoros/Projects/AI_ON_FPGA/ProjectsRF/RFanalysis0


KeyboardInterrupt: 